## Bayesian Classification
### By Team 29 - Abhigyan Chattopadhyay (EE19B146), Nihal John George (EE19B131)

In [6]:
# Imports

import pandas as pd
import os

In [7]:
# Path constants
data_path = './Data/'
team_num = '29'
pr_type = 'linsep'

total_path = data_path+team_num+'-'+pr_type+'/'

In [9]:
# Load data
train = pd.read_csv(total_path+'trian.txt', header=None, names=['x1', 'x2', 'y'])
dev = pd.read_csv(total_path+'dev.txt', header=None, names=['x1', 'x2', 'y'])

train.head()

,x1,x2,y
0,2.7092,3.1619,1
1,1.7126,5.1063,1
2,3.4792,5.0759,1
3,1.5750,1.7354,1
4,2.6529,4.9581,1


In [11]:
# Exploring some stats of train data
train.describe()

,x1,x2,y
count,1050.000000,1050.000000,1050.000000
mean,5.441763,8.334312,2.000000
std,6.747959,4.472929,0.816886
min,-5.043000,-3.270100,1.000000
25%,0.290765,3.984775,1.000000
50%,2.113650,8.920550,2.000000
75%,13.587250,12.211000,3.000000
max,18.492000,17.246000,3.000000


In [ ]:
# Bayes with Cov diff for all classes
# 
# For each class i, find the 2x2 cov matrix of x1 and x2
# cov_i = [[Ex1x1, Ex1x2], 
#        [Ex2x1, Ex2x2]]
# Find expectations from dataset by doing average of those 
# products over examples with y = i. This is MLE of cov given unimodal Gaussian assumption
# Now need to find mean meu vector for each class. 
# Find average of x1s and x2s for each class, this is MLE of mean
# given unimodal Gaussian assumption

# Now with mean and cov for each class, we can form classifier.
# Equations of discriminant functions for different cases given in Duda (2001) Ch 2, pg 19

# Case 1: Bayes with same cov for all classes - Find cov matrix without considering class
# Case 2: Bayes with diff cov - Discussed above
# Case 3: Naive Bayes with cov = sigma^2 I - Find variance of all feature values, across features.
# Case 4: Naive Bayes with cov same for all classes - cov matrix is diagonal (since naive), but diagonal values could be different. So find variance of x1, find variance of x2
# Case 5: Naive Bayes with cov diff for all classes - cov matrix is diagonal (since naive), but diagonal values could be diff, and the matrix is diff for each class

# Expected results
# Case 1: Hyperplane
# Case 2: Hyperquadratic
# Case 3: Hyperplane
# Case 4: Hyperplane
# Case 5: Hyperquadratic

In [ ]:
# Finding mean of each class


In [ ]:
# Finding class-agnostic, feature-agnostic covariance (Case 3)
 

In [ ]:
# Finding class-agnostic, feature-specific covariance (Case 1, 4)

In [ ]:
# Finding class-specific, feature-specific covariance (Case 2, 5)

In [ ]:
# Set cov matrices for the 5 cases

In [ ]:
# Plot PDFs of each class for 5 cases, constant density curves and eigenvectors (plots #1,#3 in QP)

In [ ]:
# Apply Bayes rule, get classifier. Takes an input vector, returns predicted class

In [ ]:
# Discriminant functions for 5 cases

In [ ]:
# Plot discriminant functions for 5 cases (plot #2 in QP)

In [ ]:
# Test on Dev Set

In [ ]:
# Dev set confusion matrix for 5 cases (plot #4 in QP)

In [ ]:
# ROC, DET curves for 5 cases in 1 plot (plot #5 in QP)